In [8]:
## Write scripts to load data from Alphavantage to Pandas Dataframe
import numpy as np
import pandas as pd
import requests
from matplotlib import pyplot
import json

company_data = {}

# companies is supposed to be a list of symbols of companies but im not too sure whether to physically create the list of 500 symbols or a way to access it from the API.
# companies = get_sp500_tickers()
companies = ["IBM"]

categories = ["1. open", "2. high", "3. low", "4. close", "5. volume"]

for company in companies:

    company_data[company] = generate_daily_augmented(generate_daily(company), categories)
    
    
    # Supposed to seperate the company data in a cleaner format and store the company as a key, and its data/specifics as values

df = pd.DataFrame(company_data)
print(df)
# print(df.plot())
# pyplot.show()





KeyError: 'Time Series (Daily)'

In [ ]:
import requests
import bs4 as bs
import lxml

#obtain all S&P symbols
def get_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})

    tickers = []

    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)

    tickers = [s.replace('\n', '') for s in tickers]

    print(tickers)
    return tickers

get_sp500_tickers()

In [7]:
def generate_daily(stocks):

    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&outputsize=compact&apikey=1QPXCY5WU9WFE7N6'.format(stocks)
    r = requests.get(url)
    data = r.json()

    return data

In [2]:
# what is categories supposed to do?
# trying to turn return a dictionary with the key as the date and the value as an array of 5 indices (open, high, low, close, volume)
def generate_daily_augmented(stocks, categories):
    cats = {}
    for i in stocks['Time Series (Daily)']:
        for cat in categories:
            cats[i] = cats.get(i, "") + " " + stocks['Time Series (Daily)'][i][cat] 
        
         
    return cats